In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, SGDClassifier, 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.pipeline import make_pipeline

Realizaremos una búsqueda aleatorio de los mejores hiperparámetros, esto es debido a que una búsqueda más exhaustiva para un dataset tan grande es demasiado ineficiente.

In [ ]:
def regresionLogistica(X, y, n, v_cruzada, seed):
    log_reg = LogisticRegression(max_iter=500)
    parametros_log_reg = {'C': np.logspace(-4, 4, 20), 'penalty': ['l1', 'l2'], 'solver': ['liblinear']}
    busqueda_log_reg = RandomizedSearchCV(estimator=log_reg, param_distributions=parametros_log_reg, n_iter=n, \
                                      scoring='f1_weighted', cv=v_cruzada, verbose=1, random_state=seed, n_jobs=-1)
    
    busqueda_log_reg.fit(X, y)
    parametros = busqueda_log_reg.best_params_
    print(f'Los mejores parámetros son: {parametros} con una puntuación de {busqueda_log_reg.best_score_}')

    modelo = LogisticRegression(**parametros)
    scores = cross_val_score(modelo, X, y, cv=v_cruzada, scoring='f1_weighted')
    return scores.mean()


def SGDC(X, y, n, v_cruzada, seed):
    sgd = SGDClassifier()
    parametros_sgdc = {'loss': ['hinge', 'squared_error', 'log_loss'], 'alpha': np.logspace(-5, 0, 10),\
                       'max_iter': [100, 200], 'tol': [1e-3, 1e-4]}
    busqueda_sgdc = RandomizedSearchCV(estimator=sgd, param_distributions=parametros_sgdc, n_iter=n,\
                                       scoring='f1_weighted', cv=v_cruzada, verbose=1, random_state=seed, n_jobs=-1)
    
    busqueda_sgdc.fit(X, y)
    parametros = busqueda_sgdc.best_params_
    print(f'Los mejores parámetros son: {parametros} con una puntuación de {busqueda_sgdc.best_score_}')

    modelo = SGDClassifier(**parametros)
    scores = cross_val_score(modelo, X, y, cv=v_cruzada, scoring='f1_weighted')
    return scores.mean()


def arbolDecision(X, y, n, v_cruzada, seed):
    dtc = DecisionTreeClassifier()
    parametros_dtc = {'max_depth': list(np.arange(20, 50, 2)), 'min_samples_split': np.arange(10, 20, 2), 'criterion': ['gini', 'entropy']}
    busqueda_dtc = RandomizedSearchCV(estimator=dtc, param_distributions=parametros_dtc, n_iter=n, scoring='f1_weighted',\
                                      cv=v_cruzada, verbose=1, random_state=seed, n_jobs=-1)
    
    busqueda_dtc.fit(X, y)
    parametros = busqueda_dtc.best_params_
    print(f'Los mejores parámetros son: {parametros} con una puntuación de {busqueda_dtc.best_score_}')

    modelo = DecisionTreeClassifier(**parametros)
    scores = cross_val_score(modelo, X, y, cv=v_cruzada, scoring='f1_weighted')
    return scores.mean()


def bosqueAleatorio(X, y, n, v_cruzada, seed):
    rf = RandomForestClassifier()
    parametros_rf = {'n_estimators': np.arange(100, 200, 20), 'max_features': ['sqrt', 'log2']+list(range(1, 10, 3)), \
                     'max_depth': list(np.arange(5, 20, 5)), 'min_samples_split': np.arange(5, 20, 5)}
    busqueda_rf = RandomizedSearchCV(estimator=rf, param_distributions=parametros_rf, n_iter=n, scoring='f1_weighted',\
                                   cv = v_cruzada, verbose=1, random_state=seed, n_jobs=-1)
    
    busqueda_rf.fit(X, y)
    parametros = busqueda_rf.best_params_
    print(f'Los mejores parámetros son: {parametros} con una puntuación de {busqueda_rf.best_score_}')

    modelo = RandomForestClassifier(**parametros)
    scores = cross_val_score(modelo, X, y, cv=v_cruzada, scoring='f1_weighted')
    return scores.mean()


def KNN(X, y, n, v_cruzada, seed):
    knn = KNeighborsClassifier()
    parametros_knn = {'n_neighbors': np.arange(1, 15, 3), 'weights': ['uniform', 'distance'],\
                      'metric': ['euclidean', 'manhattan']}
    busqueda_knn = RandomizedSearchCV(estimator=knn, param_distributions=parametros_knn, n_iter=n,\
                                    scoring='f1_weighted', cv=v_cruzada, verbose=1, random_state=seed, n_jobs=-1)
    
    busqueda_knn.fit(X, y)
    parametros = busqueda_knn.best_params_
    print(f'Los mejores parámetros son: {parametros} con una puntuación de {busqueda_knn.best_score_}')

    modelo = KNeighborsClassifier(**parametros)
    scores = cross_val_score(modelo, X, y, cv=v_cruzada, scoring='f1_weighted')
    return scores.mean()


In [10]:
v_cruzada = 5
seed = 42
n = 30

df1_over = pd.read_csv('df tipo 1\df1_over.csv')
df1 = pd.read_csv('df tipo 1\df1.csv')

X, y = df1.drop('label', axis=1), df1['label']
X_over, y_over = df1_over.drop('label', axis=1), df1_over['label']

In [ ]:
print(regresionLogistica(X, y, n, v_cruzada, seed))
print(SGDC(X, y, n, v_cruzada, seed))
print(arbolDecision(X, y, n, v_cruzada, seed))
print(bosqueAleatorio(X, y, n, v_cruzada, seed))
print(KNN(X, y, n, v_cruzada, seed))

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'solver': 'liblinear', 'penalty': 'l1', 'C': 29.763514416313132} con una puntuación de 0.8781402610628779
0.8780411178157668
Fitting 5 folds for each of 30 candidates, totalling 150 fits


c:\Users\Teng Xin Lin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Los mejores parámetros son: {'tol': 0.001, 'max_iter': 100, 'loss': 'log_loss', 'alpha': 0.0004641588833612782} con una puntuación de 0.6300925051388532


c:\Users\Teng Xin Lin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
c:\Users\Teng Xin Lin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
c:\Users\Teng Xin Lin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
c:\Users\Teng Xin Lin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reache

0.5254557450526234
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Los mejores parámetros son: {'min_samples_split': 12, 'max_depth': 20, 'criterion': 'entropy'} con una puntuación de 0.8308873862182546
0.830266655216404
Fitting 5 folds for each of 30 candidates, totalling 150 fits


c:\Users\Teng Xin Lin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
40 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
19 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Teng Xin Lin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Teng Xin Lin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\Teng Xin Lin\AppData\Local\Programs\Python\Python311\Lib\

Los mejores parámetros son: {'n_estimators': 180, 'min_samples_split': 15, 'max_features': 'sqrt', 'max_depth': 15} con una puntuación de 0.8804392044680981
0.8801507147119816
Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\Teng Xin Lin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 20 is smaller than n_iter=30. Running 20 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
print(regresionLogistica(X_over, y_over, n, v_cruzada, seed))
print(SGDC(X_over, y_over, n, v_cruzada, seed))
print(arbolDecision(X_over, y_over, n, v_cruzada, seed))
print(bosqueAleatorio(X_over, y_over, n, v_cruzada, seed))
print(KNN(X_over, y_over, n, v_cruzada, seed))